# Downloading a pre-trained networks with torchvision.models you can

In [4]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt

import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models

# Print out the model architecture 

In [5]:
model = models.densenet121(pretrained=True)

Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to /root/.cache/torch/hub/checkpoints/densenet121-a639ec97.pth


In [6]:
model

DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu

In [4]:
pwd

'/content'

In [5]:
ls

sample_data/


In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
cd /content/drive/MyDrive/Udacity-Intro to Deep Learning-Introduction to PyTorch

/content/drive/MyDrive/Udacity-Intro to Deep Learning-Introduction to PyTorch


In [8]:
ls

'1- PyTorch basics.ipynb'
'2- An Untrained Neural Network for Recognizing MNIST Handwritten Digits in PyTorch.ipynb'
'3- A Trained Neural Network for Recognizing MNIST Handwritten Digits in PyTorch.ipynb'
'4- A trained neural network for classifying clothing images using Fashion-MNIST.ipynb'
'5- Inference, Validation and Accuracy.ipynb'
'6- Saving and Loading Models.ipynb'
'7-  Loading Image Data.ipynb'
'8- Transfer Learning (pre-trained convolutional networks on ImageNet dataset) for building a cat dog classifier.ipynb'
 Cat_Dog_data/
 Cat_Dog_data.zip
'PyTorch Tutorial.ipynb'


In [9]:
data_dir = 'Cat_Dog_data/Cat_Dog_data'

# TODO: Define transforms for the training data and testing data
train_transforms = transforms.Compose([transforms.RandomRotation(30),
                                       transforms.RandomResizedCrop(224),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406],
                                                            [0.229, 0.224, 0.225])])

test_transforms = transforms.Compose([transforms.Resize(255),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406],
                                                           [0.229, 0.224, 0.225])])

# Pass transforms in here, then run the next cell to see how the transforms look
train_data = datasets.ImageFolder(data_dir + '/train', transform=train_transforms)
test_data = datasets.ImageFolder(data_dir + '/test', transform=test_transforms)

trainloader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)
testloader = torch.utils.data.DataLoader(test_data, batch_size=64)

In [10]:
# Freeze parameters so we don't backprop through them
for param in model.parameters():
    param.requires_grad = False

from collections import OrderedDict
classifier = nn.Sequential(OrderedDict([
                          ('fc1', nn.Linear(1024, 500)),
                          ('relu', nn.ReLU()),
                          ('fc2', nn.Linear(500, 2)),
                          ('output', nn.LogSoftmax(dim=1))
                          ]))
    
model.classifier = classifier

# Comparison the speed of network training with and without GPU

In [11]:
import time

In [12]:
for device in ['cpu', 'cuda']:

    criterion = nn.NLLLoss()
    # Only train the classifier parameters, feature parameters are frozen
    optimizer = optim.Adam(model.classifier.parameters(), lr=0.001)

    model.to(device)

    for ii, (inputs, labels) in enumerate(trainloader):

        # Move input and label tensors to the GPU
        inputs, labels = inputs.to(device), labels.to(device)

        start = time.time()

        outputs = model.forward(inputs)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if ii==3:
            break
        
    print(f"Device = {device}; Time per batch: {(time.time() - start)/3:.3f} seconds")

Device = cpu; Time per batch: 4.133 seconds
Device = cuda; Time per batch: 0.007 seconds


# Train a pretrained models to classify the cat and dog images

In [13]:
# Use GPU if it's available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = models.densenet121(pretrained=True)

# Freeze parameters so we don't backprop through them
for param in model.parameters():
    param.requires_grad = False
    
model.classifier = nn.Sequential(nn.Linear(1024, 256),
                                 nn.ReLU(),
                                 nn.Dropout(0.2),
                                 nn.Linear(256, 2),
                                 nn.LogSoftmax(dim=1))

criterion = nn.NLLLoss()

# Only train the classifier parameters, feature parameters are frozen
optimizer = optim.Adam(model.classifier.parameters(), lr=0.003)

model.to(device);

In [ ]:
epochs = 1
steps = 0
running_loss = 0
print_every = 5
for epoch in range(epochs):
    for inputs, labels in trainloader:
        steps += 1
        # Move input and label tensors to the default device
        inputs, labels = inputs.to(device), labels.to(device)
        
        logps = model.forward(inputs)
        loss = criterion(logps, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        
        if steps % print_every == 0:
            test_loss = 0
            accuracy = 0
            model.eval()
            with torch.no_grad():
                for inputs, labels in testloader:
                    inputs, labels = inputs.to(device), labels.to(device)
                    logps = model.forward(inputs)
                    batch_loss = criterion(logps, labels)
                    
                    test_loss += batch_loss.item()
                    
                    # Calculate accuracy
                    ps = torch.exp(logps)
                    top_p, top_class = ps.topk(1, dim=1)
                    equals = top_class == labels.view(*top_class.shape)
                    accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
                    
            print(f"Epoch {epoch+1}/{epochs}.. "
                  f"Train loss: {running_loss/print_every:.3f}.. "
                  f"Test loss: {test_loss/len(testloader):.3f}.. "
                  f"Test accuracy: {accuracy/len(testloader):.3f}")
            running_loss = 0
            model.train()

Epoch 1/1.. Train loss: 0.740.. Test loss: 0.303.. Test accuracy: 0.880
Epoch 1/1.. Train loss: 0.381.. Test loss: 0.126.. Test accuracy: 0.964
Epoch 1/1.. Train loss: 0.210.. Test loss: 0.077.. Test accuracy: 0.977
Epoch 1/1.. Train loss: 0.264.. Test loss: 0.063.. Test accuracy: 0.979
Epoch 1/1.. Train loss: 0.205.. Test loss: 0.056.. Test accuracy: 0.981
Epoch 1/1.. Train loss: 0.159.. Test loss: 0.054.. Test accuracy: 0.980
Epoch 1/1.. Train loss: 0.188.. Test loss: 0.058.. Test accuracy: 0.980
Epoch 1/1.. Train loss: 0.278.. Test loss: 0.064.. Test accuracy: 0.976
Epoch 1/1.. Train loss: 0.263.. Test loss: 0.088.. Test accuracy: 0.971
Epoch 1/1.. Train loss: 0.303.. Test loss: 0.076.. Test accuracy: 0.973
Epoch 1/1.. Train loss: 0.156.. Test loss: 0.062.. Test accuracy: 0.981
Epoch 1/1.. Train loss: 0.148.. Test loss: 0.055.. Test accuracy: 0.980
Epoch 1/1.. Train loss: 0.148.. Test loss: 0.048.. Test accuracy: 0.982
Epoch 1/1.. Train loss: 0.186.. Test loss: 0.049.. Test accuracy